In [1]:
# Load the adjacency matrix into a numpy array.
normal_mat = np.loadtxt('../Normal.txt', dtype=int)
for i in range(0,70):
        normal_mat[i][i] = 0
G_Normal = nx.from_numpy_matrix(normal_mat)
print("Number Of Edges: ",len(G_Normal.edges()))
print("Number Of Nodes: ",len(G_Normal.nodes()))

NameError: name 'np' is not defined

In [3]:
# Load the adjacency matrix into a numpy array.
eMCI_mat = np.loadtxt('../eMCI.txt', dtype=int)
for i in range(0,70):
        eMCI_mat[i][i] = 0
G_eMCI = nx.from_numpy_matrix(eMCI_mat)
print("Number Of Edges: ",len(G_eMCI.edges()))
print("Number Of Nodes: ",len(G_eMCI.nodes()))

Number Of Edges:  1510
Number Of Nodes:  70


In [4]:
lmci_mat = np.loadtxt('../lMCI.txt', dtype=int)
#removing self loops
for i in range(0,70):
        lmci_mat[i][i] = 0
G_lMCI = nx.from_numpy_matrix(lmci_mat)
print("Number Of Edges: ",len(G_lMCI.edges()))
print("Number Of Nodes: ",len(G_lMCI.nodes()))

Number Of Edges:  1380
Number Of Nodes:  70


In [5]:
# Load the adjacency matrix into a numpy array.
ad_mat = np.loadtxt('../AD.txt', dtype=int)
for i in range(0,70):
        ad_mat[i][i] = 0
G_AD = nx.from_numpy_matrix(ad_mat)
print("Number Of Edges: ",len(G_AD.edges()))
print("Number Of Nodes: ",len(G_AD.nodes()))

Number Of Edges:  1364
Number Of Nodes:  70


## Taking complement

In [19]:
#Normal complement
G_Normal_complement = nx.complement(G_Normal)
normal_mat_complement = nx.adjacency_matrix(G_Normal_complement)


In [20]:
#eMCI complement
G_eMCI_complement = nx.complement(G_eMCI)
emci_mat_complement = nx.adjacency_matrix(G_eMCI_complement)

In [21]:
#eMCI complement
G_lMCI_complement = nx.complement(G_lMCI)
lmci_mat_complement = nx.adjacency_matrix(G_lMCI_complement)

In [22]:
#eMCI complement
G_AD_complement = nx.complement(G_AD)
ad_mat_complement = nx.adjacency_matrix(G_AD_complement)

## Making pipeline

In [6]:
def Edge_List(Graph):
    edges = list(Graph.edges)
    node_list_1 = [item[0] for item in edges]
    node_list_2 = [item[1] for item in edges]
    # combine all nodes in a list
    node_list = node_list_1 + node_list_2
    # remove duplicate items from the list
    node_list = list(dict.fromkeys(node_list))
    df = pd.DataFrame({'node_1': node_list_1, 'node_2': node_list_2})
    return df,node_list

In [7]:
edges_Normal,edgeList_Normal = Edge_List(G_Normal)
edges_EMCI,edgeList_EMCI = Edge_List(G_eMCI)
edges_LMCI,edgeList_LMCI = Edge_List(G_lMCI)
edges_AD,edgeList_AD = Edge_List(G_AD)

In [23]:
edges_Normal_complement,edgeList_Normal_complement = Edge_List(G_Normal_complement)
edges_EMCI_complement,edgeList_EMCI_complement = Edge_List(G_eMCI_complement)
edges_LMCI_complement,edgeList_LMCI_complement = Edge_List(G_lMCI_complement)
edges_AD_complement,edgeList_AD_complement = Edge_List(G_AD_complement)

In [8]:
#recreate the graph
def recreateGraph(e_df,node_list):
    G = nx.from_pandas_edgelist(e_df, "node_1", "node_2", create_using=nx.Graph())
    adj_G = nx.to_numpy_matrix(G,nodelist = node_list)
    return G,adj_G
    

In [24]:
G_Normal_complement,adj_normal_complement = recreateGraph(edges_Normal_complement,edgeList_Normal_complement)
G_EMCI_complement,adj_emci_complement = recreateGraph(edges_EMCI_complement,edgeList_EMCI_complement)
G_LMCI_complement,adj_lmci_complement = recreateGraph(edges_LMCI_complement,edgeList_LMCI_complement)
G_AD_complement,adj_ad_complement = recreateGraph(edges_AD_complement,edgeList_AD_complement)

In [ ]:
G_Normal_,adj_normal = recreateGraph(edges_Normal,edgeList_Normal)
G_EMCI,adj_emci = recreateGraph(edges_EMCI,edgeList_EMCI)
G_LMCI,adj_lmci = recreateGraph(edges_LMCI,edgeList_LMCI)
G_AD,adj_ad = recreateGraph(edges_AD,edgeList_AD)

In [10]:
def create_Dataset(Graph,G_data):
    AA = []
    PA = []
    RAI = []
    JC = []
    CSH = []
    RAISH = []
    WIC = []
    CN=[]
    for i,j in zip(G_data['node_1'],G_data['node_2']):
        AA.append(list(nx.adamic_adar_index(Graph,ebunch=[(i,j)]))[0][2])
        RAI.append(list(nx.resource_allocation_index(Graph,ebunch=[(i,j)]))[0][2])
        PA.append(list(nx.preferential_attachment(Graph,ebunch=[(i,j)]))[0][2])
        JC.append(list(nx.jaccard_coefficient(Graph,ebunch=[(i,j)]))[0][2])
        CN.append(len(list(nx.common_neighbors(Graph, i, j))))
        #CNC.append(list(nx.common_neighbor_centrality(G, ebunch=[(i, j)]))[0][2])
    G_data['AA'] = AA     
    G_data['RAI'] = RAI
    G_data['PA'] = PA
    G_data['JC'] = JC 
    G_data['CN'] = CN
   
    #df_AllNodes['CNC'] = CNC 
    return G_data

In [11]:
def create_Dataset_for_pred(Graph):
    edges = list(Graph.edges)
    node_list_1 = [item[0] for item in edges]
    node_list_2 = [item[1] for item in edges]
    # combine all nodes in a list
    node_list = node_list_1 + node_list_2

    # remove duplicate items from the list
    node_list = list(dict.fromkeys(node_list))
    G_data = pd.DataFrame()
    AA = []
    PA = []
    RAI = []
    JC = []
    CSH = []
    RAISH = []
    WIC = []
    CN=[]
    _node1 = []
    _node2 = []
    target = []
    for i in node_list:
        for j in node_list:
            if i != j:
                _node1.append(i)
                _node2.append(j)
                AA.append(list(nx.adamic_adar_index(Graph,ebunch=[(i,j)]))[0][2])
                RAI.append(list(nx.resource_allocation_index(Graph,ebunch=[(i,j)]))[0][2])
                PA.append(list(nx.preferential_attachment(Graph,ebunch=[(i,j)]))[0][2])
                JC.append(list(nx.jaccard_coefficient(Graph,ebunch=[(i,j)]))[0][2])
                CN.append(len(list(nx.common_neighbors(Graph, i, j))))
                #CNC.append(list(nx.common_neighbor_centrality(G, ebunch=[(i, j)]))[0][2])
                if Graph.has_edge(i,j)==True:
                    target.append(1)
                else:
                    target.append(0)
    G_data['node1'] = _node1
    G_data['node2'] = _node2     
    G_data['AA'] = AA     
    G_data['RAI'] = RAI
    G_data['PA'] = PA
    G_data['JC'] = JC 
    G_data['CN'] = CN
    G_data['Target'] = target
   
    #df_AllNodes['CNC'] = CNC 
    return G_data

In [12]:
def prepareDataSet(Graph,adj_G):
    edges = list(Graph.edges)
    node_list_1 = [item[0] for item in edges]
    node_list_2 = [item[1] for item in edges]
    # combine all nodes in a list
    node_list = node_list_1 + node_list_2
    # remove duplicate items from the list
    node_list = list(dict.fromkeys(node_list))
    G_df = pd.DataFrame({'node_1': node_list_1, 'node_2': node_list_2})
    # get unconnected node-pairs
    
    all_unconnected_pairs = []
    # traverse adjacency matrix
    offset = 0
    for i in tqdm(range(adj_G.shape[0])):
        for j in range(offset,adj_G.shape[1]):
            if i != j:
                if adj_G[i,j] == 0:
                    all_unconnected_pairs.append([node_list[i],node_list[j]])

        offset = offset + 1
    node_1_unlinked = [i[0] for i in all_unconnected_pairs]
    node_2_unlinked = [i[1] for i in all_unconnected_pairs]
    data = pd.DataFrame({'node_1':node_1_unlinked, 
                         'node_2':node_2_unlinked})
    # add target variable 'link'
    data['link'] = 0
    initial_node_count = len(Graph.nodes)

    G_df_temp = G_df.copy()
    # empty list to store removable links
    omissible_links_index = []
    for i in tqdm(G_df.index.values):
      # remove a node pair and build a new graph
      G_temp = nx.from_pandas_edgelist(G_df_temp.drop(index = i), "node_1", "node_2", create_using=nx.Graph())
      # check there is no spliting of graph and number of nodes is same
      if (nx.number_connected_components(G_temp) == 1) and (len(G_temp.nodes) == initial_node_count):
        omissible_links_index.append(i)
        G_df_temp = G_df_temp.drop(index = i)
    # create dataframe of removable edges
    G_df_ghost = G_df.loc[omissible_links_index]
    # add the target variable 'link'
    G_df_ghost['link'] = 1
    data = data.append(G_df_ghost[['node_1', 'node_2', 'link']], ignore_index=True)
    # drop removable edges
    G_df_partial = G_df.drop(index=G_df_ghost.index.values)
    # build graph
    G_data = nx.from_pandas_edgelist(G_df_partial, "node_1", "node_2", create_using=nx.Graph())
    return data,G_data

In [13]:
def model_(G_first,G_sec,adj_mat):
    data,G_data = prepareDataSet(G_first,adj_mat)
    df = create_Dataset(G_data,data)
    xtrain, xtest, ytrain, ytest = train_test_split(df[['AA','RAI','PA','JC','CN']], df['link'], test_size = 0.3, random_state = 35)
    lr = LogisticRegression()
    lr.fit(xtrain, ytrain)
    predictions = lr.predict(xtest)
    #roc_auc_score(ytest, predictions[:,1])
    print("Test accuracy",accuracy_score(ytest,predictions))
    first_dataset = create_Dataset_for_pred(G_first)
    second_dataset = create_Dataset_for_pred(G_sec)
    predictions = lr.predict(first_dataset[['AA','RAI','PA','JC','CN']])
    #roc_auc_score(ytest, predictions[:,1])
    print("predictions :\n",predictions)
    print("Training accuracy",accuracy_score(first_dataset['Target'],predictions))
    print("Next state prediction accuracy",accuracy_score(second_dataset['Target'],predictions))
    print("Confusion matrix: \n",confusion_matrix(second_dataset['Target'],predictions))

## Implementing pipeline on actual graph

In [15]:
model_(G_Normal,G_EMCI,adj_normal)

100%|██████████| 1490/1490 [00:08<00:00, 169.76it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6400602409638554
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.34591747146619845
Next state prediction accuracy 0.33713784021071114
Confusion matrix: 
 [[1536    0]
 [3020    0]]


In [16]:
model_(G_EMCI,G_LMCI,adj_emci)

100%|██████████| 1510/1510 [00:09<00:00, 155.54it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6445783132530121
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.33713784021071114
Next state prediction accuracy 0.3942054433713784
Confusion matrix: 
 [[1796    0]
 [2760    0]]


In [17]:
model_(G_LMCI,G_AD,adj_lmci)

100%|██████████| 1380/1380 [00:10<00:00, 135.68it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6039156626506024
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.3942054433713784
Next state prediction accuracy 0.40122914837576823
Confusion matrix: 
 [[1828    0]
 [2728    0]]


In [18]:
model_(G_Normal,G_AD,adj_normal)

100%|██████████| 1490/1490 [00:07<00:00, 190.97it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6400602409638554
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.34591747146619845
Next state prediction accuracy 0.40122914837576823
Confusion matrix: 
 [[1828    0]
 [2728    0]]


## Implementing pipeline on complemented graph

In [25]:
model_(G_Normal_complement,G_EMCI_complement,adj_normal_complement)

100%|██████████| 788/788 [00:05<00:00, 149.17it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6716867469879518
predictions :
 [0 1 1 ... 0 0 0]
Training accuracy 0.8024582967515365


ValueError: Found input variables with inconsistent numbers of samples: [4830, 4556]

In [26]:
model_(G_EMCI_complement,G_LMCI_complement,adj_emci_complement)

100%|██████████| 905/905 [00:06<00:00, 137.30it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6534090909090909
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.6252587991718427
Next state prediction accuracy 0.5714285714285714
Confusion matrix: 
 [[2760    0]
 [2070    0]]


In [27]:
model_(G_LMCI_complement,G_AD_complement,adj_lmci_complement)

100%|██████████| 1035/1035 [00:09<00:00, 114.11it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.5923295454545454
predictions :
 [0 0 0 ... 0 0 0]
Training accuracy 0.5714285714285714


ValueError: Found input variables with inconsistent numbers of samples: [4556, 4830]

In [28]:
model_(G_Normal_complement,G_AD_complement,adj_normal_complement)

100%|██████████| 788/788 [00:04<00:00, 183.72it/s]
C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy 0.6716867469879518
predictions :
 [0 1 1 ... 0 0 0]
Training accuracy 0.8024582967515365
Next state prediction accuracy 0.6549604916593503
Confusion matrix: 
 [[1696 1032]
 [ 540 1288]]
